# Automobile Dataset - Preparing data for Analysis

This notebook contains detailed steps followed for cleansing and preparing the automobile data set. For more details about the dataset and attribute information, [refer here](https://archive.ics.uci.edu/ml/datasets/automobile)

**Summary of steps**
* Data Preparation (This Notebook)
 * Cleanse non-numeric values from numeric columns
 * Convert/Integer encode text data
 * Derive new fields based on available ones




In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Read data from CSV file
autodata=pd.read_csv("./Automobile_data.txt")

#cars.info()
#cars.describe()

# Data Sanity Check 

## Find columns that need cleansing
List the count of non-numeric values in numric columns. 

In [30]:
# List of columns that are supposed to contain numeric data
cols2numeric = ['symboling','normalized-losses','wheel-base','length', 'width','height','curb-weight','engine-size','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']

print("***** Count of Non Numeric Elements - Columnwise ******")
for col in cols2numeric:
    if(pd.to_numeric(autodata[col],errors='coerce').isnull().sum() > 0):
        print(col + ": " + str(pd.to_numeric(autodata[col],errors='coerce').isnull().sum()))


***** Count of Non Numeric Elements - Columnwise ******
normalized-losses: 41
bore: 4
stroke: 4
horsepower: 2
peak-rpm: 2
price: 4


In [31]:
#List of columns that contain a "?" for missing data
colslist = list(autodata.columns)
for col in colslist:
    if('?' in autodata[col].value_counts()):
        print(col + " - " + str(autodata[col].value_counts()['?']))

normalized-losses - 41
num-of-doors - 2
bore - 4
stroke - 4
horsepower - 2
peak-rpm - 2
price - 4


**Following columns will need to be sanitized, before they can be used for analysis.**

* 1.normalized-losses
* 2.price
* 3. horsepower 
* 4. bore
* 5. stroke

** 1. normalized-losses ** 

* a. Convert "normalized-losses" column to numeric. In the process, insert NaNs where values cannot be converted to a number
* b. Under each make, if there are enough number of records with valid loss values (>=50%), find their mean value and replace NaNs.  
* c. If more than 50% records under a make have NaNs for loss value, those records have to be discarded for analysis.

In [32]:
# Dataframe for Normalised-Losses related analysis
carsnl=autodata
#Replace non-numeric characters in normalized-losses column with NaN.
carsnl['normalized-losses'] = pd.to_numeric(carsnl['normalized-losses'],errors='coerce')

#Find makes having normalized-losses as NaNs and their number of occurrances 
carsnan=carsnl[carsnl['normalized-losses'].isnull()].groupby(by='make', as_index=False).size().reset_index()
carsnan.columns=['make','nanscount']

#Find makes with count of all records under each make
carsgp = carsnl.groupby(by='make',as_index=False).size().reset_index()
carsgp.columns=['make','makecount']
carsgpnan=carsgp.merge(carsnan,on="make", how="left", suffixes=['', '_right'])

#Find makes having more than 50% records with valid normalized-losses values
mean_nan_makes = carsgpnan[carsgpnan['nanscount']*100/carsgpnan['makecount'] <= 50].reset_index()
mean_nan_makes['nlimpute']=mean_nan_makes['make']

#Find mean loss figures for those makes
nlcars=carsnl.merge(mean_nan_makes, on="make", how="left", suffixes=['','_right'])
nlmean_by_make = nlcars.groupby(by='nlimpute')['normalized-losses'].mean().reset_index()

#Replace NaN values with the calculated mean 

carsr=carsnl.merge(nlmean_by_make, how="left", left_on="make", right_on="nlimpute", suffixes=['', '_right'])

#cars.describe()
#np.where(cars['normalized-losses'].isnull(), cars['normalized-losses_right'], cars['normalized-losses'])          

carsr['normalized-losses'] = np.where(carsr['normalized-losses'].isnull(), carsr['normalized-losses_right'], carsr['normalized-losses'])          
#carsnl=False

cars = carsr.loc[carsr['normalized-losses'].isnull()==False].copy()

print("Column normalized-losses has " + str(cars['normalized-losses'].count()) + " values after sanitizing")

Column normalized-losses has 189 values after sanitizing


**2. price**

Calculate the average price per make and impute the missing price values with the respective make's average price. 

In [33]:
cars['price'] = pd.to_numeric(cars['price'],errors='coerce')
mean_price_by_make = cars.groupby(by=['make'])['price'].mean()
mean_price_by_make = mean_price_by_make.reset_index()
 
# Merge mean_price_by_make (Rigt) with cars dataframe (Left)

cars = cars.merge(mean_price_by_make,on='make', suffixes=['', '_right'])
cars['price'] = np.where(cars['price'].isnull(), cars['price_right'], cars['price'])
#cars[['make','price' ]]

In [ ]:
**3. horsepower **
Convert to numeric

In [34]:
cars['horsepower'] = pd.to_numeric(cars['horsepower'], errors="coerce")

Drop all temporary columns that were added/merged in the process of data cleansing

In [35]:
del cars['normalized-losses_right']
del cars['price_right']

Remove records having number of doors as "?"

In [36]:
cars = cars[cars['num-of-doors'] != '?']

## Integer encode Text Columns - Convert to ordinal values
Following columns are converted to numeric, by interpreting the string values:
* num-of-doors
* num-of-cylinders

In [37]:
def calc_num_doors (doorstr):
    if(doorstr == 'two'):
       return 2 
    if (doorstr =='four'):
       return 4
    return 0
       
cars['num_doors'] = cars['num-of-doors'].apply(lambda x: calc_num_doors (x))

def calc_num_cylinders(cylstr):
    cyls = {'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}
    if(cylstr in cyls.keys()):
        return cyls[cylstr]
    return 0

cars['num_cylinders'] = cars['num-of-cylinders'].apply(lambda x: calc_num_cylinders(x))

## Derive columns 
* 1. Convert the Engine size from cubic inches to cubic centimeters (cc) for easier understanding. To convert cubic inch to cubic centimeter, divide cubic inch value by 0.061024
* 2. Calculate power to weight ratio of vehicles. Power to weight ratio shows the performance of a car

In [38]:
cars['engine_size_cc'] = cars['engine-size'].apply(lambda x: int(x/0.061024))
cars['power2weight_ratio'] = pd.to_numeric(cars['horsepower'], errors="coerce")/cars['curb-weight']

### Remove rows having non-numeric bore and stroke

In [39]:
cars = cars[cars['bore'] != '?']
cars = cars[cars['stroke'] != '?']
cars.describe()

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,horsepower,city-mpg,highway-mpg,price,num_doors,num_cylinders,engine_size_cc,power2weight_ratio
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,0.737705,124.274446,99.145355,174.287978,65.909836,54.009836,2542.054645,125.196721,10.185355,100.295082,25.666667,31.098361,12709.858834,3.202186,4.349727,2051.038251,0.038940
std,1.207599,35.718009,5.979826,12.574952,2.116007,2.328635,518.033610,37.826884,4.078702,33.713080,6.338729,6.813138,7438.115517,0.982034,0.843910,619.911154,0.007719
min,-2.000000,65.000000,86.600000,141.100000,60.300000,49.400000,1488.000000,61.000000,7.000000,48.000000,14.000000,16.000000,5118.000000,2.000000,3.000000,999.000000,0.019936
25%,0.000000,95.000000,94.500000,166.300000,64.000000,52.500000,2128.000000,97.500000,8.500000,70.000000,20.500000,25.000000,7649.000000,2.000000,4.000000,1597.000000,0.034139
50%,1.000000,119.000000,97.200000,173.400000,65.500000,54.300000,2405.000000,110.000000,9.000000,94.000000,25.000000,31.000000,9988.000000,4.000000,4.000000,1802.000000,0.037475
75%,2.000000,153.000000,102.400000,184.600000,66.900000,55.700000,2964.500000,141.000000,9.400000,116.000000,30.500000,36.500000,16214.000000,4.000000,4.000000,2310.000000,0.043704
max,3.000000,256.000000,120.900000,208.100000,72.000000,59.800000,4066.000000,308.000000,23.000000,200.000000,49.000000,54.000000,45400.000000,4.000000,8.000000,5047.000000,0.063715


## Check if any column has ? 

In [40]:
colslist = list(cars.columns)
for col in colslist:
    if('?' in cars[col].value_counts()):
        print(col + " - " + str(autodata[col].value_counts()['?']))

## Replace Drive-wheels with descriptive values

In [43]:
cars.loc[cars['drive-wheels']=='4wd', 'drive-wheels']='Four Wheel Drive'
cars.loc[cars['drive-wheels']=='fwd', 'drive-wheels']='Front Wheel Drive'
cars.loc[cars['drive-wheels']=='rwd', 'drive-wheels']='Rear Wheel Drive'

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,nlimpute,num_doors,num_cylinders,engine_size_cc,power2weight_ratio
0,2,164.0,audi,gas,std,four,sedan,Front Wheel Drive,front,99.8,...,102,5500,24,30,13950.0,audi,4,4,1786,0.043646
1,2,164.0,audi,gas,std,four,sedan,Four Wheel Drive,front,99.4,...,115,5500,18,22,17450.0,audi,4,5,2228,0.040722
2,2,161.0,audi,gas,std,two,sedan,Front Wheel Drive,front,99.8,...,110,5500,19,25,15250.0,audi,2,5,2228,0.043877
3,1,158.0,audi,gas,std,four,sedan,Front Wheel Drive,front,105.8,...,110,5500,19,25,17710.0,audi,4,5,2228,0.038678
4,1,161.0,audi,gas,std,four,wagon,Front Wheel Drive,front,105.8,...,110,5500,19,25,18920.0,audi,4,5,2228,0.037238


## Store Cleaned data to CSV file

In [44]:
cars.to_csv('./cardata_cleaned.csv')